# Example of MRSI fitting on the command line

This notebook demos the process of fitting an MRSI scan using the command line scripts included in FSL-MRS.
### Contents:  
- [1. Reconstruction, Processing and Data Conversion](#1.-Reconstruction,-Processing-and-Data-Conversion)
- [2. Fitting](#2.-Fitting)
- [3. Fitting](#3.-Fitting)
- [4. Visualisation of fit](#4.-Visualisation-of-fit)

Will Clarke  
June 2020  
University of Oxford

## 1. Reconstruction, processing and data conversion

MRSI reconstruction (from k-space data) can be highly specialised depending on the sequence. For example reconstruction of non-cartesian trajectories (e.g. spirals or concentric-rings) requires regridding or use of the NUFFT. Due to the specialised nature of MRSI reconstruction FSL-MRS does not contain tools for this step.

Simmilarly post-processing of MRSI data is commonly done as part of the reconstruction process. Though many of the processing tools in FSL-MRS can be run on MRSI data they have not been created with MRSI in mind.

This example therefore assumes that you are able to provide reconstructed and processed data ready for fitting. Data can be converted to NIfTI from the Siemens DICOM format using spec2nii. The authors of FSL-MRS and spec2nii are happy to add supported formats if example data and interpretation is provided.

## 2. Tissue segmentation

Tissue segmentation is required to have meaningful scaling of water referenced metabolite concentrations. mrsi_segment will produce three files, corresponding to the GM, WM and CSF FAST tissue segmentations registered to the MRSI volume.

Run tissue segmentation on the packaged T1 data and mask using the SVS voxel. Here we provide a (partial) .anat file produced by [fsl_anat](https://fsl.fmrib.ox.ac.uk/fsl/fslwiki/fsl_anat) to speed up execution.

This step requires an installation of FSL compatible with fslpy.

In [4]:
%%capture
%sx mkdir MRSI
%sx mrsi_segment -o MRSI -a example_data/example_mrsi/T1.anat example_data/example_mrsi/FID_Metab.nii.gz

## 3. Fitting

In [13]:
%sx fsl_mrsi --data example_data/example_mrsi/mrsi.nii.gz \
             --basis example_data/example_mrsi/3T_slaser_32vespa_1250.BASIS \
             --output MRSI/example_mrsi_fit \
             --mask example_data/example_mrsi/mask.nii.gz \
             --h2o example_data/example_mrsi/wref.nii.gz \
             --tissue_frac MRSI/mrsi_seg_wm.nii.gz MRSI/mrsi_seg_gm.nii.gz MRSI/mrsi_seg_csf.nii.gz \
             --add_MM \
             --TE 30 \
             --baseline_order 4 \
             --combine PCho GPC --combine Cr PCr --combine NAA NAAG --combine Glu Gln --combine Glc Tau \
             --ignore Gly HG \
             --overwrite

['',
 '',
 '',
 '-----------------------------------------------------',
 '',
 '',
 '',
 '',
 '     _____ ____  _          __  __ ____  ____ ___ ',
 '    |  ___/ ___|| |        |  \\/  |  _ \\/ ___|_ _|',
 '    | |_  \\___ \\| |   _____| |\\/| | |_) \\___ \\| | ',
 '    |  _|  ___) | |__|_____| |  | |  _ < ___) | | ',
 '    |_|   |____/|_____|    |_|  |_|_| \\_\\____/___|',
 '                    ',
 '    ',
 '',
 '',
 '',
 '-----------------------------------------------------',
 '',
 '',
 '',
 '',
 '--->> Start fitting',
 '',
 '',
 '    Algorithm = [Newton]',
 '',
 '    Initialise with average fit',
 '    Parallelising over 8 workers ',
 '    96 Voxels remaining    ',
 '    96 Voxels remaining    ',
 '    96 Voxels remaining    ',
 '    96 Voxels remaining    ',
 '    96 Voxels remaining    ',
 '    96 Voxels remaining    ',
 '    90 Voxels remaining    ',
 '    72 Voxels remaining    ',
 '    72 Voxels remaining    ',
 '    72 Voxels remaining    ',
 '    72 Voxels remaining    ',
 '

## 4. Visualisation of fit
Now take a look at the outputs 

In [29]:
%sx ls MRSI/example_mrsi_fit

['concs', 'fit', 'fit_avg.png', 'options.txt', 'qc', 'uncertainties']

In [30]:
%%capture
%sx pythonw $(which fsleyes) example_data/example_mrsi/T1.anat/T1.nii.gz MRSI/example_mrsi_fit/fit/fit.nii.gz MRSI/example_mrsi_fit/concs/molarity/NAA+NAAG.nii.gz